## Importante Pacotes e Carregando o Dataset

In [7]:
import pandas as pd





pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
df_original = pd.read_csv("Dados/dados_coletados10k.csv")
df_original.head()

,Contrato,Idade,Sexo,Valor_Renda,UF_Cliente,Perc_Juros,Prazo_Emprestimo,Data_Contratacao,Prazo_Restante,VL_Emprestimo,VL_Emprestimo_ComJuros,QT_Total_Parcelas_Pagas,QT_Total_Parcelas_Pagas_EmDia,QT_Total_Parcelas_Pagas_EmAtraso,Qt_Renegociacao,Estado_Civil,Escolaridade,Possui_Patrimonio,VL_Patrimonio,QT_Parcelas_Atraso,QT_Dias_Atraso,Saldo_Devedor,Total_Pago,Possivel_Fraude
0,322068935715,43,M,5800.0,SP,23.0,200,2022-08-01,193,80000.0,92000.0,15,9,1,0,DIVORCIADO,Nenhum,N,0.0,0,NaN,91759.97,7717.08,Nao
1,322068936715,22,M,2000.0,MG,20.0,100,2022-08-01,89,50000.0,57500.0,10,10,0,0,SOLTEIRO(A),NaN,N,0.0,4,103.0,53981.18,6756.59,Nao
2,322068938715,35,M,4000.0,BA,18.0,100,2022-08-01,85,100000.0,115000.0,15,5,1,0,CASADO (A),NaN,N,0.0,0,NaN,101150.02,17929.06,Nao
3,322068939715,20,M,1800.0,MG,20.0,100,2022-08-01,89,30000.0,34500.0,1,1,0,0,SOLTEIRO(A),NaN,N,0.0,13,376.0,36208.10,369.10,Sim
4,322068940715,53,M,2800.0,MG,20.0,100,2022-08-01,87,60000.0,69000.0,16,16,0,0,CASADO (A),NaN,N,0.0,0,NaN,60984.00,11845.24,Nao


# Analise Exploratória - Visão Geral
<br>
Aqui o objetivo desta analise é ter uma visão geral do conjunto de dados, compreendendo seu tamanho, variaveis, 
tipo de dados e período dos dados coletados.

In [10]:
# Verificando a estrutura da base de dados
df_original.shape

(9517, 24)

In [12]:
# Informações sobre a base de dados
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9517 entries, 0 to 9516
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Contrato                          9517 non-null   int64  
 1   Idade                             9517 non-null   int64  
 2   Sexo                              9517 non-null   object 
 3   Valor_Renda                       9517 non-null   float64
 4   UF_Cliente                        9517 non-null   object 
 5   Perc_Juros                        9517 non-null   float64
 6   Prazo_Emprestimo                  9517 non-null   int64  
 7   Data_Contratacao                  9517 non-null   object 
 8   Prazo_Restante                    9517 non-null   int64  
 9   VL_Emprestimo                     9517 non-null   float64
 10  VL_Emprestimo_ComJuros            9517 non-null   float64
 11  QT_Total_Parcelas_Pagas           9517 non-null   int64  
 12  QT_Tot

In [14]:
# Verificando valores nulos de cada coluna
df_original.isnull().sum()

Contrato                               0
Idade                                  0
Sexo                                   0
Valor_Renda                            0
UF_Cliente                             0
Perc_Juros                             0
Prazo_Emprestimo                       0
Data_Contratacao                       0
Prazo_Restante                         0
VL_Emprestimo                          0
VL_Emprestimo_ComJuros                 0
QT_Total_Parcelas_Pagas                0
QT_Total_Parcelas_Pagas_EmDia          0
QT_Total_Parcelas_Pagas_EmAtraso       0
Qt_Renegociacao                        0
Estado_Civil                           0
Escolaridade                        7105
Possui_Patrimonio                      0
VL_Patrimonio                          0
QT_Parcelas_Atraso                     0
QT_Dias_Atraso                      3594
Saldo_Devedor                          0
Total_Pago                             0
Possivel_Fraude                        0
dtype: int64

- Possuimos valores nulos nas coluna **Escolaridade** e **QT_Dias_Atraso**

In [15]:
# Estatísticas da base de dados
df_original.describe()

,Contrato,Idade,Valor_Renda,Perc_Juros,Prazo_Emprestimo,Prazo_Restante,VL_Emprestimo,VL_Emprestimo_ComJuros,QT_Total_Parcelas_Pagas,QT_Total_Parcelas_Pagas_EmDia,QT_Total_Parcelas_Pagas_EmAtraso,Qt_Renegociacao,VL_Patrimonio,QT_Parcelas_Atraso,QT_Dias_Atraso,Saldo_Devedor,Total_Pago
count,9.517000e+03,9517.000000,9.517000e+03,9517.000000,9517.000000,9517.000000,9517.000000,9517.000000,9517.000000,9517.000000,9517.000000,9517.000000,9517.000000,9517.000000,5923.000000,9517.000000,9517.000000
mean,3.220782e+11,38.738678,8.325404e+03,19.645729,107.425344,104.583482,81881.885048,94164.167805,7.893139,4.108122,1.597247,1.242093,10.927813,5.353788,249.789802,90560.267866,8166.206714
std,5.434161e+06,12.674802,1.218621e+05,3.820652,62.488802,68.568209,94138.059627,108258.768571,5.165751,4.040356,2.313021,1.165580,1025.262221,5.320100,123.221927,111050.541232,16697.758870
min,3.220689e+11,6.000000,4.500000e+02,7.000000,15.000000,0.000000,3500.000000,4025.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000
25%,3.220733e+11,29.000000,2.300000e+03,18.000000,60.000000,51.000000,20000.000000,23000.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,162.000000,18546.850000,1202.040000
50%,3.220785e+11,37.000000,3.400000e+03,20.000000,80.000000,80.000000,50000.000000,57500.000000,9.000000,2.000000,1.000000,1.000000,0.000000,4.000000,284.000000,45375.300000,3949.910000
75%,3.220826e+11,46.000000,5.000000e+03,22.000000,190.000000,185.000000,100000.000000,115000.000000,13.000000,6.000000,2.000000,2.000000,0.000000,11.000000,344.000000,119721.250000,8302.010000
max,3.220876e+11,91.000000,8.000080e+06,28.000000,240.000000,227.000000,500000.000000,575000.000000,35.000000,35.000000,14.000000,9.000000,100000.000000,15.000000,435.000000,625000.000000,396385.000000


In [41]:
# valores unicos de cada coluna

for coluna in df_original.columns:
    print(f"{coluna} : {len(df_original[coluna].unique())}")

Contrato : 9517
Idade : 74
Sexo : 2
Valor_Renda : 855
UF_Cliente : 27
Perc_Juros : 21
Prazo_Emprestimo : 36
Data_Contratacao : 110
Prazo_Restante : 79
VL_Emprestimo : 61
VL_Emprestimo_ComJuros : 61
QT_Total_Parcelas_Pagas : 24
QT_Total_Parcelas_Pagas_EmDia : 24
QT_Total_Parcelas_Pagas_EmAtraso : 15
Qt_Renegociacao : 10
Estado_Civil : 6
Escolaridade : 6
Possui_Patrimonio : 2
VL_Patrimonio : 3
QT_Parcelas_Atraso : 16
QT_Dias_Atraso : 16
Saldo_Devedor : 7654
Total_Pago : 7022
Possivel_Fraude : 2


In [42]:
# Avaliando o maior e menor valor da variavel Valor_Renda
print('Maior Renda:', df_original['Valor_Renda'].max())
print('Menor Renda:', df_original['Valor_Renda'].min())

Maior Renda: 8000080.0
Menor Renda: 450.0


In [43]:
# Avaliando o maior e menor valor da variavel QT_Dias_Atraso
print('Maior quantidade de dias atraso: ', df_original['QT_Dias_Atraso'].max())
print('Menor quantidade de dias atraso: ', df_original['QT_Dias_Atraso'].min())

Maior quantidade de dias atraso:  435.0
Menor quantidade de dias atraso:  11.0


In [44]:
# Avaliando o maior e menor valor da variavel Prazo_Restante
print('Maior quantidade de dias restante: ', df_original['Prazo_Restante'].max())
print('Menor quantidade de dias restante: ', df_original['Prazo_Restante'].min())

Maior quantidade de dias restante:  227
Menor quantidade de dias restante:  0


In [62]:
# Quantidade de dias em atraso
df_original.groupby(['QT_Dias_Atraso']).size()

QT_Dias_Atraso
11.0     532
41.0     255
71.0     190
103.0    205
133.0    176
162.0    206
194.0    241
225.0    352
251.0    471
284.0    546
315.0    718
344.0    930
376.0    686
406.0    413
435.0      2
dtype: int64

In [60]:
# Quantidade de dias em atraso
df_original['QT_Dias_Atraso'].value_counts(ascending=False)

344.0    930
315.0    718
376.0    686
284.0    546
11.0     532
251.0    471
406.0    413
225.0    352
41.0     255
194.0    241
162.0    206
103.0    205
71.0     190
133.0    176
435.0      2
Name: QT_Dias_Atraso, dtype: int64